*TP réalisé par Mehdi Rouan-Serik dans le cadre du cours d'Alexandrina Rogozan*
# TP4 : Implémentation du Chiffrement RSA

Ce notebook vous guidera à travers l'implémentation complète de l'algorithme RSA (Rivest-Shamir-Adleman), un système de chiffrement asymétrique fondamental en cryptographie.

## Objectifs
- Comprendre les principes mathématiques du RSA
- Implémenter les fonctions utilitaires
- Générer et utiliser des clés RSA
- Chiffrer et déchiffrer des messages

**Note** : Ce TP utilise des nombres premiers de petite taille pour la démonstration. En production, des nombres beaucoup plus grands (2048 bits ou plus) sont utilisés.

Sourdrille Nathan

## 1) Importation des bibliothèques

Nous avons besoin du module `random` pour générer des nombres premiers aléatoires.

In [30]:
import random
import math

## 2) Génération d'un nombre premier

Implémentez la fonction `generate_prime()` qui :

- Génère un nombre aléatoire entre $2^{15}$ et $2^{16}$
- Teste si ce nombre est premier en vérifiant qu'il n'a aucun diviseur entre $2$ et $\sqrt{n}$
- Retourne le premier nombre premier trouvé

In [31]:
def is_prime(nb):
    for i in range(2, int(math.sqrt(nb))):
        if nb%i == 0:
            return False
    return True

In [32]:
def generate_prime():
    """
    Génère un nombre premier aléatoire.
    
    Cette fonction génère aléatoirement des entiers entre 2^15 et 2^16
    et teste leur primalité en utilisant le test de divisibilité.
    
    Paramètres :
        Aucun
        
    Retour :
        int : Un nombre premier généré aléatoirement
    """
    is_prime_nb=False
    nb = 0
    while not is_prime_nb:
        nb = random.randint(2**15, 2**16)
        is_prime_nb = is_prime(nb)
    return nb

In [33]:
print(f"Nombre aléatoire généré : {generate_prime()}")

Nombre aléatoire généré : 33071


## 3) Génération de deux nombres premiers distincts

Implémentez la fonction `generate_two_distinct_primes()` qui :

- Génère deux nombres premiers $p$ et $q$ en utilisant `generate_prime()`
- S'assure que $p$ et $q$ sont différents
- Retourne le tuple $(p, q)$

In [34]:
def generate_two_distinct_primes():
    """
    Génère deux nombres premiers distincts.
    
    Cette fonction génère deux nombres premiers différents p et q,
    essentiels pour la création des clés RSA.
    
    Paramètres :
        Aucun
        
    Retour :
        tuple : (p, q) où p et q sont deux entiers premiers distincts
    """
    p = 0
    q = 0
    while p == q:
        p = generate_prime()
        q = generate_prime()
    return (p,q)

In [35]:
print(f"Tuple de nombres premiers générés : {generate_two_distinct_primes()}")

Tuple de nombres premiers générés : (60763, 54493)


## 4) Calcul du PGCD avec l'algorithme d'Euclide

Implémentez la fonction `euclide(a, b)` qui :

- Calcule le Plus Grand Commun Diviseur (PGCD) de deux nombres
- Utilise l'algorithme d'Euclide classique
- Retourne le PGCD

In [36]:
def euclide(a, b):
    """
    Calcule le Plus Grand Commun Diviseur (PGCD) de deux nombres.
    
    Implémente l'algorithme d'Euclide classique qui utilise la propriété :
    PGCD(a, b) = PGCD(b, a mod b) jusqu'à ce que b = 0.
    
    Paramètres :
        a (int) : Premier entier
        b (int) : Deuxième entier
        
    Retour :
        int : Le PGCD de a et b
    """
    while b != 0 : 
        a, b = b, a%b 
    return a

In [37]:
print(f"PGCD entre 28 et 14 : {euclide(28, 14)}")
a, b = generate_two_distinct_primes()
print(f"PGCD entre {a} et {b} : {euclide(a, b)}")

PGCD entre 28 et 14 : 14
PGCD entre 40933 et 44263 : 1


## 5) Génération de l'exposant public $e$

Implémentez la fonction `get_e(phi)` qui :

- Cherche le plus petit nombre $e$ impair tel que $\textrm{PGCD}(e, \varphi(n)) = 1$
- Commence à $e = 3$ et incrémente par $2$ pour rester impair
- Utilise la fonction `euclide()` pour vérifier la condition
- Retourne l'exposant public $e$

In [38]:
def get_phi(p, q):
    return (p - 1) * (q - 1)

In [39]:
def get_e(phi):
    """
    Génère l'exposant public e pour la clé RSA.
    
    Recherche le plus petit nombre impair e tel que PGCD(e, φ(n)) = 1.
    L'exposant public doit être copremier avec φ(n).
    
    Paramètres :
        phi (int) : L'indicatrice d'Euler φ(n) = (p-1) × (q-1)
        
    Retour :
        int : L'exposant public e (généralement une petite valeur comme 3, 5, 7, ...)
    """
    e = 3
    while euclide(e,phi) != 1:
        e += 2
    return e

In [40]:
p, q = generate_two_distinct_primes()
phi = get_phi(p, q)
print(f"p : {p}\nq : {q}\nphi : {phi}")
print(f"Exposant e généré : {get_e(phi)}")

p : 45541
q : 39293
phi : 1789357680
Exposant e généré : 7


## 6) Algorithme d'Euclide étendu

Implémentez la fonction `extended_euclide(a, b)` qui :

- Calcule le PGCD de $a$ et $b$ (comme Euclide classique)
- Calcule également les coefficients de Bézout $(x, y)$ tels que : $$ax + by = \textrm{PGCD}(a, b)$$
- Retourne le tuple $(x, y, \textrm{pgcd})$

**Indications** :
- Maintenez deux paires de coefficients : $(x_0, x_1)$ et $(y_0, y_1)$
- Initialisez : $x_0=1, x_1=0, y_0=0, y_1=1$
- À chaque itération : $q = a // b$, puis mettez à jour les coefficients
- Les mises à jour : $x_1, x_0 = x_0 - qx_1, x_1$ et $y_1, y_0 = y_0 - q*y_1, y_1$
- Retournez $x_0$, $y_0$ et $a$ (le PGCD final)

In [41]:
def extended_euclide(a, b):
    """
    Calcule le PGCD et les coefficients de Bézout.
    
    Implémente l'algorithme d'Euclide étendu qui trouve des entiers x et y
    tels que : a*x + b*y = PGCD(a, b).
    
    Cette fonction est essentielle pour calculer l'inverse modulo d dans RSA.
    
    Paramètres :
        a (int) : Premier entier
        b (int) : Deuxième entier
        
    Retour :
        tuple : (x, y, pgcd) où a*x + b*y = pgcd
    """
    x0, x1 = 1, 0
    y0, y1 = 0, 1

    while b != 0:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1

    return x0, y0, a

In [42]:
p, q = generate_two_distinct_primes()
x, y, pgcd = extended_euclide(p, q)
print(f"p : {p}\nq : {q}")
print(f"PGCD avec euclide etendu : {pgcd}")
print(f"x : {x}\ny : {y}\na*x+b*y : {p*x + q*y}")
print(f"L'algorithme fonctionne : {p*x + q*y == pgcd}")

p : 38953
q : 47353
PGCD avec euclide etendu : 1
x : 7988
y : -6571
a*x+b*y : 1
L'algorithme fonctionne : True


## 7) Conversion chaîne / bytes

Implémentez la fonction `bytes_from_str(msg: str)` qui :

- Convertit une chaîne de caractères en octets UTF-8
- Retourne une liste d'entiers (les valeurs des octets)

**Indications** :
- Utilisez la méthode `.encode('utf-8')` pour convertir la chaîne en bytes
- Utilisez `list()` pour convertir les bytes en liste d'entiers
- Chaque caractère UTF-8 peut être représenté par un ou plusieurs octets

On encode la chaîne en UTF-8 car c’est le standard universel de représentation des caractères. Chaque caractère devient un octet (ou plusieurs pour les caractères non ASCII). Cette conversion est nécessaire car RSA ne chiffre que des nombres.

In [43]:
def bytes_from_str(msg: str) -> list:
    """
    Convertit une chaîne de caractères en liste d'octets UTF-8.
    
    Cette fonction encode le message en UTF-8 et retourne
    une liste d'entiers représentant chaque octet.
    
    Paramètres :
        msg (str) : La chaîne de caractères à convertir
        
    Retour :
        list : Liste d'entiers (0-255) représentant les octets UTF-8
    """
    return list(msg.encode('utf-8'))

In [44]:
msg = "bonjour"
bytes_msg = bytes_from_str(msg)
print(f"\"{msg}\" converti en bytes : {bytes_msg}")

"bonjour" converti en bytes : [98, 111, 110, 106, 111, 117, 114]


## 8) Conversion liste de bytes / chaîne

Implémentez la fonction `string_from_bytes(byte_list: list)` qui :

- Convertit une liste d'octets en chaîne de caractères
- Utilise le décodage UTF-8
- Retourne la chaîne de caractères originale

**Indications** :
- Utilisez `bytes(byte_list)` pour convertir la liste en objet bytes
- Utilisez la méthode `.decode('utf-8')` pour décoder en chaîne de caractères
- C'est l'inverse de `bytes_from_str()`

On retransforme la liste d’octets en objet bytes puis on la décode en UTF-8 pour récupérer la chaîne originale. UTF-8 garantit que la conversion est réversible.

In [45]:
def string_from_bytes(byte_list: list) -> str:
    """
    Convertit une liste d'octets en chaîne de caractères UTF-8.
    
    Cette fonction est l'inverse de bytes_from_str() : elle
    convertit une liste d'entiers en chaîne de caractères.
    
    Paramètres :
        byte_list (list) : Liste d'entiers (0-255) représentant les octets
        
    Retour :
        str : La chaîne de caractères décodée en UTF-8
    """
    return bytes(byte_list).decode('utf-8')

In [46]:
bytes_msg = [98, 111, 110, 106, 111, 117, 114]
msg = string_from_bytes(bytes_msg)
print(f"{bytes_msg} decode en str : {msg}")

[98, 111, 110, 106, 111, 117, 114] decode en str : bonjour


## 9) Chiffrement RSA d'un bloc

Implémentez la fonction `rsa_encrypt(m, n, e)` qui :

- Calcule le message chiffré $c = m^e\;\textrm{mod}\; n$
- Utilise l'exponentiation modulaire
- Retourne le message chiffré $c$

**Indications** :
- Utilisez `pow(m, e, n)` pour calculer $m^e\;\textrm{mod}\; n$ de façon efficace
- Cette fonction chiffre un seul bloc (un octet dans notre cas)
- $m$ doit être strictement inférieur à $n$

Chiffrement RSA: c = mᵉ mod n.

In [47]:
def rsa_encrypt(m, n, e):
    """
    Chiffre un bloc de message avec RSA.
    
    Calcule c = m^e mod n où m est le message clair,
    e est l'exposant public et n est le module RSA.
    
    Paramètres :
        m (int) : Le bloc de message à chiffrer (doit être < n)
        n (int) : Le module RSA (produit p × q)
        e (int) : L'exposant public
        
    Retour :
        int : Le bloc de message chiffré c
    """
    assert m < n, "Le bloc à chiffrer doit etre strictement inferieur au module"
    return pow(m, e, n)

## 10) Déchiffrement RSA d'un bloc

Implémentez la fonction `rsa_decrypt(c, n, d)` qui :

- Calcule le message déchiffré $m = c^d \; \textrm{mod} \; n$
- Utilise l'exponentiation modulaire
- Retourne le message en clair $m$

**Indications** :
- Utilisez `pow(c, d, n)` pour calculer $c^d \;\textrm{mod}\; n$ de façon efficace
- Cette fonction déchiffre un seul bloc (un octet dans notre cas)
- $d$ est l'exposant privé (secret)

Déchiffrement RSA : m = cᵈ mod n.

In [48]:
def rsa_decrypt(c, n, d):
    """
    Déchiffre un bloc de message chiffré avec RSA.
    
    Calcule m = c^d mod n où c est le message chiffré,
    d est l'exposant privé et n est le module RSA.
    
    Paramètres :
        c (int) : Le bloc de message chiffré
        n (int) : Le module RSA (produit p × q)
        d (int) : L'exposant privé (secret)
        
    Retour :
        int : Le bloc de message en clair m
    """
    return pow(c, d, n)

## 11) Chiffrement d'une liste de bytes

Implémentez la fonction `bytes_chiffrement(byte_list: list, n: int, e: int)` qui :

- Chiffre chaque octet de la liste en utilisant `rsa_encrypt()`
- Retourne une liste d'octets chiffrés

**Indications** :
- Chaque octet est chiffré indépendamment
- Les octets chiffrés peuvent être plus grands que $255$

In [49]:
def bytes_chiffrement(byte_list: list, n: int, e: int) -> list:
    """
    Chiffre une liste d'octets avec RSA.
    
    Chiffre chaque octet individuellement en utilisant
    la clé publique (n, e).
    
    Paramètres :
        byte_list (list) : Liste d'entiers (octets à chiffrer)
        n (int) : Le module RSA
        e (int) : L'exposant public
        
    Retour :
        list : Liste d'entiers chiffrés
    """
    bytes_chiffres = []
    for b in byte_list:
        bytes_chiffres.append(rsa_encrypt(b, n, e))
    return bytes_chiffres

## 12) Déchiffrement d'une liste de bytes

Implémentez la fonction `bytes_dechiffrement(byte_list: list, n: int, d: int)` qui :

- Déchiffre chaque octet de la liste en utilisant `rsa_decrypt()`
- Retourne une liste d'octets déchiffrés

**Indications** :
- Chaque octet chiffré est déchiffré indépendamment
- Les octets déchiffrés doivent être entre $0$ et $255$

In [50]:
def bytes_dechiffrement(byte_list: list, n: int, d: int) -> list:
    """
    Déchiffre une liste d'octets chiffrés avec RSA.
    
    Déchiffre chaque octet individuellement en utilisant
    la clé privée (n, d).
    
    Paramètres :
        byte_list (list) : Liste d'entiers (octets chiffrés)
        n (int) : Le module RSA
        d (int) : L'exposant privé (secret)
        
    Retour :
        list : Liste d'entiers déchiffrés
    """
    bytes_dechiffres = []
    for b in byte_list:
        bytes_dechiffres.append(rsa_decrypt(b, n, d))
    return bytes_dechiffres

## 13) Génération complète des clés RSA

Implémentez la fonction `generate_rsa_keys()` qui :

- Génère deux nombres premiers distincts $p$ et $q$
- Calcule $n = p \times q$
- Calcule $\varphi(n) = (p-1) \times (q-1)$
- Génère l'exposant public $e$
- Calcule l'exposant privé $d = e^{-1} \;\textrm{mod}\; \varphi(n)$ (inverse modulaire)
- Retourne un dictionnaire avec les clés publique et privée

In [51]:
def generate_rsa_keys():
    """
    Génère une paire de clés RSA publique et privée.
    
    Effectue les étapes suivantes :
    1. Génère deux nombres premiers distincts p et q
    2. Calcule le module RSA : n = p × q
    3. Calcule l'indicatrice d'Euler : φ(n) = (p-1) × (q-1)
    4. Choisit l'exposant public e (copremier avec φ(n))
    5. Calcule l'exposant privé d (inverse modulaire de e mod φ(n))
    6. Affiche les paramètres générés
    
    Paramètres :
        Aucun
        
    Retour :
        dict : Dictionnaire contenant :
            - 'pub' : tuple (n, e) - clé publique
            - 'priv' : d - clé privée (exposant privé)
    """
    p, q = generate_two_distinct_primes()
    n = p * q
    phi = get_phi(p, q)
    e = get_e(phi)
    d = extended_euclide(e, phi)[0]
    dico_cles = {'pub': (n, e), 'priv': d}
    print(f"p : {p}\nq : {q}\nphi : {phi}\ne : {e}\nd : {d}")
    print("dico cles : ", dico_cles)
    
    return dico_cles

## 14) Test et Démonstration

Une fois toutes les fonctions implémentées, testez-les avec le code suivant :

### Test 1 : Chiffrement d'un nombre simple

Commençons par tester le chiffrement RSA sur un seul octet :

In [52]:
# Génération des clés RSA
keys = generate_rsa_keys()
print(f"Clé publique (n, e) : {keys['pub']}")
print(f"Clé privée d : {keys['priv']}")

# Test avec un nombre simple (un octet)
n, e = keys['pub']
d = keys['priv']

m = 65  # Représente le caractère 'A'
print(f"\nMessage simple : {m}")

c = rsa_encrypt(m, n, e)
print(f"Message chiffré : {c}")

m_decrypted = rsa_decrypt(c, n, d)
print(f"Message déchiffré : {m_decrypted}")

print(f"Succès : {m == m_decrypted}")

p : 38197
q : 46199
phi : 1764578808
e : 5
d : -705831523
dico cles :  {'pub': (1764663203, 5), 'priv': -705831523}
Clé publique (n, e) : (1764663203, 5)
Clé privée d : -705831523

Message simple : 65
Message chiffré : 1160290625
Message déchiffré : 65
Succès : True


### Test 2 : Chiffrement d'un texte court

Testons maintenant le chiffrement d'une chaîne de caractères complète :

In [53]:
# Test du chiffrement/déchiffrement d'un message texte
message = "Bonjour ITI4 !"
print(f"Message original : {message}")

# Conversion en bytes
byte_list = bytes_from_str(message)
print(f"Bytes : {byte_list}")
print(f"Nombre d'octets : {len(byte_list)}")

# Chiffrement
encrypted = bytes_chiffrement(byte_list, n, e)
print(f"\nBytes chiffrés : {encrypted}")

# Déchiffrement
decrypted = bytes_dechiffrement(encrypted, n, d)
print(f"Bytes déchiffrés : {decrypted}")

# Conversion en chaîne
message_decoded = string_from_bytes(decrypted)
print(f"\nMessage déchiffré : {message_decoded}")

# Vérification
print(f"Succès : {message == message_decoded}")

Message original : Bonjour ITI4 !
Bytes : [66, 111, 110, 106, 111, 117, 114, 32, 73, 84, 73, 52, 32, 33]
Nombre d'octets : 14

Bytes chiffrés : [1252332576, 968612724, 223131173, 1029613355, 968612724, 748521921, 1607513794, 33554432, 308408390, 652793018, 308408390, 380204032, 33554432, 39135393]
Bytes déchiffrés : [66, 111, 110, 106, 111, 117, 114, 32, 73, 84, 73, 52, 32, 33]

Message déchiffré : Bonjour ITI4 !
Succès : True


### Test 3 : Chiffrement d'un fichier texte

Enfin, testons le chiffrement d'un contenu de fichier plus volumineux :

In [54]:
# Créer un contenu de fichier texte plus long
texte_fichier = """Ceci est un exemple de texte plus long qui pourrait provenir d'un fichier.
RSA est un algorithme de chiffrement asymétrique très important en cryptographie.
Les clés publique et privée permettent de sécuriser les communications.
Cette implémentation est à titre éducatif et utilise de petits nombres premiers."""

print(f"Contenu du fichier (original) :\n{texte_fichier}\n")
print(f"Longueur du texte : {len(texte_fichier)} caractères\n")

# Conversion en bytes
byte_list_file = bytes_from_str(texte_fichier)
print(f"Nombre d'octets à chiffrer : {len(byte_list_file)}\n")

# Chiffrement
encrypted_file = bytes_chiffrement(byte_list_file, n, e)
print(f"Fichier chiffré (premiers 20 blocs) : {encrypted_file[:20]}\n")

# Déchiffrement
decrypted_file = bytes_dechiffrement(encrypted_file, n, d)

# Conversion en chaîne
texte_decoded = string_from_bytes(decrypted_file)
print(f"Contenu du fichier (déchiffré) :\n{texte_decoded}\n")

# Vérification
if texte_fichier == texte_decoded:
    print("✓ Succès : Le fichier a été correctement chiffré puis déchiffré !")
else:
    print("✗ Erreur : Le texte déchiffré ne correspond pas à l'original")
    print(f"Différences : {len(texte_fichier)} vs {len(texte_decoded)} caractères")

Contenu du fichier (original) :
Ceci est un exemple de texte plus long qui pourrait provenir d'un fichier.
RSA est un algorithme de chiffrement asymétrique très important en cryptographie.
Les clés publique et privée permettent de sécuriser les communications.
Cette implémentation est à titre éducatif et utilise de petits nombres premiers.

Longueur du texte : 309 caractères

Nombre d'octets à chiffrer : 317

Fichier chiffré (premiers 20 blocs) : [1350125107, 1686784486, 686584484, 410173204, 33554432, 1686784486, 702276642, 1592121343, 33554432, 748521921, 223131173, 33554432, 1686784486, 177915158, 1686784486, 1268933925, 1741448005, 575975144, 1686784486, 33554432]

Contenu du fichier (déchiffré) :
Ceci est un exemple de texte plus long qui pourrait provenir d'un fichier.
RSA est un algorithme de chiffrement asymétrique très important en cryptographie.
Les clés publique et privée permettent de sécuriser les communications.
Cette implémentation est à titre éducatif et utilise de peti

faire signature et verification

In [55]:
def signature_rsa(m, d, n):
    return pow(m, d, n)

In [56]:
def verification_rsa(s, e, n):
    return pow(s, e, n)

In [57]:
msg = 145

signature = signature_rsa(msg, d, n)
print(f"Signature RSA : {signature}")

verification = verification_rsa(signature, e, n)
print(f"Verification RSA : {verification}")

Signature RSA : 1758758384
Verification RSA : 145


Ainsi, la vérification RSA correspond bien à la signature ce qui signifie que seul le propriétaire de la clé privée d a pu créer la signature.

En conclusion, on a bien recréé le mécanisme de signature et vérification RSA.

Ce TP nous a permis d’implémenter entièrement le chiffrement RSA, depuis les fonctions utilitaires jusqu’au chiffrement et déchiffrement d’un fichier complet. Nous avons appliqué les étapes fondamentales du RSA (génération des clés, exponentiation modulaire, conversions entre chaînes et octets) et vérifié la cohérence de l’ensemble à travers plusieurs tests.
Même si les valeurs utilisées sont volontairement petites pour faciliter les calculs, cette implémentation illustre concrètement le fonctionnement d’un système de chiffrement asymétrique et montre comment RSA garantit la confidentialité grâce à la difficulté de factoriser de grands nombres.